## US National Parks & Yelp! Reviews

In [2]:
# dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
from pprint import pprint

# API and DB Credentials
# from keys import db_cred
from keys import gkey



### Extract Park Data from JSON


In [3]:
json_file = "resources/data.json"
parks_df = pd.read_json(json_file)
parks_df.head()

,area,coordinates,date_established_readable,date_established_unix,description,image,nps_link,states,title,id,visitors,world_heritage_site
0,"{'acres': '49,057.36', 'square_km': '198.5'}","{'latitude': 44.35, 'longitude': -68.21}","February 26, 1919",-1604599200,Covering most of Mount Desert Island and other...,"{'url': 'acadia.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/acad/index.htm,"[{'id': 'state_maine', 'title': 'Maine'}]",Acadia,park_acadia,"3,303,393",False
1,"{'acres': '8,256.67', 'square_km': '33.4'}","{'latitude': -14.25, 'longitude': -170.68}","October 31, 1988",594280800,The southernmost National Park is on three Sam...,"{'url': 'american-samoa.jpg', 'attribution': '...",https://www.nps.gov/npsa/index.htm,"[{'id': 'state_american-samoa', 'title': 'Amer...",American Samoa,park_american-samoa,"28,892",False
2,"{'acres': '76,678.98', 'square_km': '310.3'}","{'latitude': 38.68, 'longitude': -109.57}","November 12, 1971",58773600,"This site features more than 2,000 natural san...","{'url': 'arches.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/arch/index.htm,"[{'id': 'state_utah', 'title': 'Utah'}]",Arches,park_arches,"1,585,718",False
3,"{'acres': '242,755.94', 'square_km': '982.4'}","{'latitude': 43.75, 'longitude': -102.5}","November 10, 1978",279525600,"The Badlands are a collection of buttes, pinna...","{'url': 'badlands.jpg', 'attribution': 'PixelB...",https://www.nps.gov/badl/index.htm,"[{'id': 'state_south-dakota', 'title': 'South ...",Badlands,park_badlands,"996,263",False
4,"{'acres': '801,163.21', 'square_km': '3,242.2'}","{'latitude': 29.25, 'longitude': -103.25}","June 12, 1944",-806439600,Named for the prominent bend in the Rio Grande...,"{'url': 'big-bend.jpg', 'attribution': 'PixelB...",https://www.nps.gov/bibe/index.htm,"[{'id': 'state_texas', 'title': 'Texas'}]",Big Bend,park_big-bend,"388,290",False


### Transform Park Data

In [65]:
natl_parks=parks_df.drop(columns=['date_established_unix','image', 'nps_link','states'])
natl_parks.set_index('title',drop=True)
natl_parks.head()

,area,coordinates,date_established_readable,description,title,id,visitors,world_heritage_site
0,"{'acres': '49,057.36', 'square_km': '198.5'}","{'latitude': 44.35, 'longitude': -68.21}","February 26, 1919",Covering most of Mount Desert Island and other...,Acadia,park_acadia,"3,303,393",False
1,"{'acres': '8,256.67', 'square_km': '33.4'}","{'latitude': -14.25, 'longitude': -170.68}","October 31, 1988",The southernmost National Park is on three Sam...,American Samoa,park_american-samoa,"28,892",False
2,"{'acres': '76,678.98', 'square_km': '310.3'}","{'latitude': 38.68, 'longitude': -109.57}","November 12, 1971","This site features more than 2,000 natural san...",Arches,park_arches,"1,585,718",False
3,"{'acres': '242,755.94', 'square_km': '982.4'}","{'latitude': 43.75, 'longitude': -102.5}","November 10, 1978","The Badlands are a collection of buttes, pinna...",Badlands,park_badlands,"996,263",False
4,"{'acres': '801,163.21', 'square_km': '3,242.2'}","{'latitude': 29.25, 'longitude': -103.25}","June 12, 1944",Named for the prominent bend in the Rio Grande...,Big Bend,park_big-bend,"388,290",False


In [28]:
park_list = []

for target_search in natl_parks['title']:
    park_list.append(target_search)

park_list

['Acadia',
 'American Samoa',
 'Arches',
 'Badlands',
 'Big Bend',
 'Biscayne',
 'Black Canyon of the Gunnison',
 'Bryce Canyon',
 'Canyonlands',
 'Capitol Reef',
 'Carlsbad Caverns',
 'Channel Islands',
 'Congaree',
 'Crater Lake',
 'Cuyahoga Valley',
 'Death Valley',
 'Denali',
 'Dry Tortugas',
 'Everglades',
 'Gates of the Arctic',
 'Glacier',
 'Glacier Bay',
 'Grand Canyon',
 'Grand Teton',
 'Great Basin',
 'Great Sand Dunes',
 'Great Smoky Mountains',
 'Guadalupe Mountains',
 'Haleakala',
 'Hawaii Volcanoes',
 'Hot Springs',
 'Isle Royale',
 'Joshua Tree',
 'Katmai',
 'Kenai Fjords',
 'Kings Canyon',
 'Kobuk Valley',
 'Lake Clark',
 'Lassen Volcanic',
 'Mammoth Cave',
 'Mesa Verde',
 'Mount Rainier',
 'North Cascades',
 'Olympic',
 'Petrified Forest',
 'Pinnacles',
 'Redwood',
 'Rocky Mountain',
 'Saguaro',
 'Sequoia',
 'Shenandoah',
 'Theodore Roosevelt',
 'Virgin Islands',
 'Voyageurs',
 'Wind Cave',
 'Wrangell–St. Elias',
 'Yellowstone',
 'Yosemite',
 'Zion']

### Extract google places Reviews


In [66]:
# use Park DataFrame for names
# target_search = "acadia national park"


parks_dict = {}

for target_search in natl_parks['title']:
    params = {    
        "input": target_search + " national park",
        "inputtype": "textquery",
        "key": gkey
    }

    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

    response = requests.get(base_url, params=params)

    google_parks_data = response.json()
    park_id = google_parks_data['candidates'][0]['place_id']
    parks_dict[target_search] = park_id


In [67]:
parks_dict

{'Acadia': 'ChIJJSmiDrKjrkwRhFVV_A4i32I',
 'American Samoa': 'ChIJRx3UY-WEpnERNJZ__Rw4Hy8',
 'Arches': 'ChIJUaoNhhr2yoARlcQo0WnqQk8',
 'Badlands': 'ChIJIyAFGAyHfocR8Hfm2jFeaGk',
 'Big Bend': 'ChIJK-_lS3AX8oYRD4WDJ7x9RmQ',
 'Biscayne': 'ChIJK4Wkv7jb2YgRv7LTOSIRIRI',
 'Black Canyon of the Gunnison': 'ChIJ8Wo3SfxeP4cRvIuts5LWMT8',
 'Bryce Canyon': 'ChIJLevDAsZrNYcRBm2svvvY6Ws',
 'Canyonlands': 'ChIJqaYYRe7hR4cRquYCxalSpBU',
 'Capitol Reef': 'ChIJU6LnB_8ASocRB_9PSFPsO94',
 'Carlsbad Caverns': 'ChIJW9e4xBN544YRvbI7vfc91G4',
 'Channel Islands': 'ChIJFXm2LM5S6IARzDo6cliV5PU',
 'Congaree': 'ChIJBwa8iRQ1_4gRmiPbgxV26CE',
 'Crater Lake': 'ChIJOePlQAgXxlQRo-MvReHyK5A',
 'Cuyahoga Valley': 'ChIJdfUpIkjeMIgRquWefEq-yco',
 'Death Valley': 'ChIJR4qudndLx4ARVLDye3zwycw',
 'Denali': 'ChIJBRbQiP_zzVYRpTYZExU7tcY',
 'Dry Tortugas': 'ChIJV_YGBNFVzogRoR0zV_0OsVs',
 'Everglades': 'ChIJ2wVsejCo0IgRlzEvdlY-4A8',
 'Gates of the Arctic': 'ChIJbcXUoz4vKFERYkhcRDyuSp0',
 'Glacier': 'ChIJVVVVVRWQaFMR7F3FFSK8Fq8',


In [53]:
google_parks_data = response.json()
p_id = google_parks_data['candidates'][0]['place_id']
print(p_id)

ChIJ2fhEiNDqyoAR9VY2qhU6Lnw


In [57]:
# possible .iterrows
for key in parks_dict:
    print(key, parks_dict[key])

Acadia ChIJJSmiDrKjrkwRhFVV_A4i32I
American Samoa ChIJRx3UY-WEpnERNJZ__Rw4Hy8
Arches ChIJUaoNhhr2yoARlcQo0WnqQk8
Badlands ChIJIyAFGAyHfocR8Hfm2jFeaGk
Big Bend ChIJK-_lS3AX8oYRD4WDJ7x9RmQ
Biscayne ChIJK4Wkv7jb2YgRv7LTOSIRIRI
Black Canyon of the Gunnison ChIJ8Wo3SfxeP4cRvIuts5LWMT8
Bryce Canyon ChIJLevDAsZrNYcRBm2svvvY6Ws
Canyonlands ChIJqaYYRe7hR4cRquYCxalSpBU
Capitol Reef ChIJU6LnB_8ASocRB_9PSFPsO94
Carlsbad Caverns ChIJW9e4xBN544YRvbI7vfc91G4
Channel Islands ChIJFXm2LM5S6IARzDo6cliV5PU
Congaree ChIJBwa8iRQ1_4gRmiPbgxV26CE
Crater Lake ChIJOePlQAgXxlQRo-MvReHyK5A
Cuyahoga Valley ChIJdfUpIkjeMIgRquWefEq-yco
Death Valley ChIJR4qudndLx4ARVLDye3zwycw
Denali ChIJBRbQiP_zzVYRpTYZExU7tcY
Dry Tortugas ChIJV_YGBNFVzogRoR0zV_0OsVs
Everglades ChIJ2wVsejCo0IgRlzEvdlY-4A8
Gates of the Arctic ChIJbcXUoz4vKFERYkhcRDyuSp0
Glacier ChIJVVVVVRWQaFMR7F3FFSK8Fq8
Glacier Bay ChIJKTncO2TDAVQR4F7rb5QiqMQ
Grand Canyon ChIJFU2bda4SM4cRKSCRyb6pOB8
Grand Teton ChIJd8WCU2GrU1MRYZXka9xjvIY
Great Basin ChIJAw3M1yVcsY

In [60]:

# print(yosemite_data["result"]["name"])
# print(yosemite_data["result"]["rating"])
# print(yosemite_data["result"]["user_ratings_total"])

# MAKE 3 LISTS INSTEAD OF THE DATAFRAME (6.API's, Day 3, Activity 5)
park_title = []
park_name = []
park_rating = []
park_total_ratings = []
# park_details_df = pd.DataFrame(columns=['title', 'rating', 'user_ratings_total']) 


for key in parks_dict:

    place_id = parks_dict[key]

    params2 = {    
        "place_id": place_id,
        "key": gkey
    }

    base_detail_url= "https://maps.googleapis.com/maps/api/place/details/json"

    response2 = requests.get(base_detail_url, params=params2)

    park_details = response2.json()

    park_title.append(key)
    park_name.append(park_details["result"]["name"])
    park_rating.append(park_details["result"]["rating"])
    park_total_ratings.append(park_details["result"]["user_ratings_total"])



,title,rating,user_ratings_total


In [50]:
yosemite_data=response2.json()
pprint(yosemite_data)

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'Maine',
                                    'short_name': 'ME',
                                    'types': ['administrative_area_level_1',
                                              'political']},
                                   {'long_name': 'United States',
                                    'short_name': 'US',
                                    'types': ['country', 'political']}],
            'adr_address': '<span class="region">Maine</span>, <span '
                           'class="country-name">USA</span>',
            'business_status': 'OPERATIONAL',
            'formatted_address': 'Maine, USA',
            'formatted_phone_number': '(207) 288-3338',
            'geometry': {'location': {'lat': 44.3385559, 'lng': -68.2733346},
                         'viewport': {'northeast': {'lat': 44.40370240000001,
                                                    'lng': -68.1591412},
                

In [36]:
print(yosemite_data["result"]["name"])
print(yosemite_data["result"]["rating"])
print(yosemite_data["result"]["user_ratings_total"])

Yosemite National Park
4.8
34183


### Transform Yelp! Data

### Create Database Connection

### Load DataFrames into Database